[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/daclip-uir-colab/blob/main/daclip_uir_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/daclip-uir
%cd /content/daclip-uir

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/daclip-uir/resolve/main/daclip_ViT-B-32-2023-09_b768x4_lr3e-5_e50_zeroadd.pt -d /content/daclip-uir/pretrained -o daclip_ViT-B-32-2023-09_b768x4_lr3e-5_e50_zeroadd.pt

!pip install -q einops ema-pytorch ftfy lmdb

In [ ]:
%cd /content/daclip-uir/universal-image-restoration

import torch
from PIL import Image
import open_clip

checkpoint = '/content/daclip-uir/daclip_ViT-B-32-2023-09_b768x4_lr3e-5_e50_zeroadd.pt'
model, preprocess = open_clip.create_model_from_pretrained('daclip_ViT-B-32', pretrained=checkpoint)
tokenizer = open_clip.get_tokenizer('ViT-B-32')

image = preprocess(Image.open("/content/test.jpg")).unsqueeze(0)
degradations = ['motion-blurry','hazy','jpeg-compressed','low-light','noisy','raindrop','rainy','shadowed','snowy','uncompleted']
text = tokenizer(degradations)

with torch.no_grad(), torch.cuda.amp.autocast():
    text_features = model.encode_text(text)
    image_features, degra_features = model.encode_image(image, control=True)
    degra_features /= degra_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * degra_features @ text_features.T).softmax(dim=-1)
    index = torch.argmax(text_probs[0])

print(f"Task: {degradations[index]} - {text_probs[0][index]}")